In [ ]:
import nltk; nltk.download('stopwords')
!pip install pyLDAvis==2.1.2
!python3 -m spacy download en_core_web_md

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, ldaseqmodel
import spacy
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
import logging

In [ ]:
from nltk.corpus import stopwords
#spacy.stopwords?
stop_words = stopwords.words('english')
# stop_words.extend(['number'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df = pd.read_csv('df_pr.csv',error_bad_lines=False, engine="python")
df = df.sort_values(by=['publicationyear'], ascending=True).reset_index()
print(df)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-4-db33b0cc6c8d>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('df_pr.csv',error_bad_lines=False, engine="python")


      index                                              title  \
0      1335  The Seagram Spirits and Wine Group Realigns It...   
1       465  Brown-Forman Acquires Interest in Australian D...   
2      1851  Seagram Appoints John Preston Executive Vice P...   
3       946                     McCormick Sale Finalized Today   
4      1852  Seagram Realigns Leadership of the House of Se...   
...     ...                                                ...   
2923   2521  Jameson® Irish Whiskey Turns St. Patrick's Day...   
2924   2468  JOHNNIE WALKER COLLABORATES WITH STREETWEAR AR...   
2925   2469  GRAN CORAMINO, THE TEQUILA BRAND BY KEVIN HART...   
2926   2470  Crown Royal and Country Music Star Lainey Wils...   
2927      6  Milestone Brands Strengthens Strategic Positio...   

        publicationtitle  publicationyear  \
0     prnewswire;newyork             1992   
1     prnewswire;newyork             1992   
2     prnewswire;newyork             1992   
3     prnewswire;newyork   

In [ ]:
time_slices = []
slice_years = [1990, 2000, 2010, 2020]   # enter start years for desired time slices
for n in range(len(slice_years)-1):
  sliced_df = df[(df['publicationyear'] >= slice_years[n]) & (df['publicationyear'] < slice_years[n+1])]
  time_slices.append(len(sliced_df))
time_slices.append(len(df[df['publicationyear'] >= slice_years[-1]]))

print(time_slices)

[45, 535, 1754, 594]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
data = df.finalprocessedfulltext.values.tolist()

data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
data = [re.sub('\s+', ' ', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:3: DeprecationWarning: invalid escape sequence '\S'
<>:4: DeprecationWarning: invalid escape sequence '\s'
<>:3: DeprecationWarning: invalid escape sequence '\S'
<>:4: DeprecationWarning: invalid escape sequence '\s'
<ipython-input-6-f04fab237391>:3: DeprecationWarning: invalid escape sequence '\S'
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
<ipython-input-6-f04fab237391>:4: DeprecationWarning: invalid escape sequence '\s'
  data = [re.sub('\s+', ' ', sent) for sent in data]


['seagram spirits wine group business unit seagram company ltd. nyse vo today '
 'announced realigned management structure global brands establishing separate '
 'whisky division chivas glenlivet division division responsible strategic '
 'direction business development global marketing seagram scotch whisky brands '
 'worldwide production activities scotland responsible international '
 'development canadian american whiskies outside north america included whisky '
 'portfolio flagship brands chivas regal royal salute glenlivet passport glen '
 'grant scotch whiskies seagram v.o. crown royal canadian whiskies roses '
 'bourbon reorganization result year long management brand analysis undertaken '
 'hubert millet president seagram global brands millet assumed additional '
 'responsibility developing appropriate long term management structure seagram '
 'global brands march NUMBER effort analyzed developed strategic action plan '
 'marketing group north american scotch whisky brands foc

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['seagram', 'spirits', 'wine', 'group', 'business', 'unit', 'seagram', 'company', 'ltd', 'nyse', 'vo', 'today', 'announced', 'realigned', 'management', 'structure', 'global', 'brands', 'establishing', 'separate', 'whisky', 'division', 'chivas', 'glenlivet', 'division', 'division', 'responsible', 'strategic', 'direction', 'business', 'development', 'global', 'marketing', 'seagram', 'scotch', 'whisky', 'brands', 'worldwide', 'production', 'activities', 'scotland', 'responsible', 'international', 'development', 'canadian', 'american', 'whiskies', 'outside', 'north', 'america', 'included', 'whisky', 'portfolio', 'flagship', 'brands', 'chivas', 'regal', 'royal', 'salute', 'glenlivet', 'passport', 'glen', 'grant', 'scotch', 'whiskies', 'seagram', 'crown', 'royal', 'canadian', 'whiskies', 'roses', 'bourbon', 'reorganization', 'result', 'year', 'long', 'management', 'brand', 'analysis', 'undertaken', 'hubert', 'millet', 'president', 'seagram', 'global', 'brands', 'millet', 'assumed', 'additio

In [ ]:
#invalidate grams that wrap around

bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['seagram', 'spirits', 'wine', 'group', 'business', 'unit', 'seagram', 'company', 'ltd', 'nyse', 'vo', 'today', 'announced', 'realigned', 'management', 'structure', 'global', 'brands', 'establishing', 'separate', 'whisky', 'division', 'chivas', 'glenlivet', 'division', 'division', 'responsible', 'strategic', 'direction', 'business', 'development', 'global', 'marketing', 'seagram', 'scotch', 'whisky', 'brands', 'worldwide', 'production', 'activities', 'scotland', 'responsible', 'international', 'development', 'canadian', 'american', 'whiskies', 'outside', 'north_america', 'included', 'whisky', 'portfolio', 'flagship', 'brands', 'chivas_regal_royal_salute', 'glenlivet', 'passport', 'glen_grant', 'scotch', 'whiskies', 'seagram', 'crown_royal_canadian', 'whiskies', 'roses', 'bourbon', 'reorganization', 'result', 'year', 'long', 'management', 'brand', 'analysis', 'undertaken', 'hubert', 'millet', 'president', 'seagram', 'global', 'brands', 'millet', 'assumed', 'additional', 'responsibility'

In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
data_words_nostops = remove_stopwords(data_words)

data_words_bigrams = make_bigrams(data_words_nostops)

nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])
#try other models? transformer-based?

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['spirit', 'wine', 'group', 'today', 'announce', 'realigned', 'management', 'structure', 'global', 'brand', 'establish', 'separate', 'whisky', 'division', 'division', 'division', 'responsible', 'strategic', 'direction', 'business', 'development', 'global', 'brand', 'worldwide', 'production', 'activity', 'responsible', 'international', 'development', 'canadian', 'american', 'whisky', 'include', 'portfolio', 'flagship', 'brand', 'chivas_regal', 'royal_salute', 'whisky', 'canadian', 'whisky', 'rose', 'bourbon', 'reorganization', 'result', 'year', 'long', 'management', 'brand', 'analysis', 'undertake', 'assume', 'additional', 'responsibility', 'develop', 'appropriate', 'long_term', 'management', 'structure', 'brand', 'number', 'effort', 'analyze', 'develop', 'strategic', 'action', 'plan', 'marketing', 'group', 'focus', 'study', 'ensure', 'position', 'profitably', 'grow', 'expand', 'market', 'share', 'key', 'whisky', 'brand', 'world', 'head', 'new', 'division', 'appoint', 'serve', 'base', 

In [ ]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INF
ldaseq = ldaseqmodel.LdaSeqModel('dtm-linux64'
                                  corpus=corpus,
                                 id2word=id2word,
                                 time_slice=time_slices,
                                 num_topics=15,
                                 passes=5,
                                 em_min_iter=2,
                                 em_max_iter=5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
INFO:gensim.models.ldamodel:using symmetric eta at 0.06666666666666667
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online (multi-pass) LDA training, 15 topics, 5 passes over the supplied corpus of 2928 documents, updating model once every 2000 documents, evaluating perplexity every 2928 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #2000/2928
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 2928 documents
INFO:gensim.models.ldamodel:topic #14 (0.010): 0.053*"number" + 0.010

In [ ]:
ldaseq.print_topics()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[('product', 0.02095030452782966),
  ('brand', 0.019365625033827473),
  ('consumer', 0.01188300476132692),
  ('natural', 0.011019528777973162),
  ('bottle', 0.009271406184444058),
  ('packaging', 0.008931069990240504),
  ('category', 0.008128170595421114),
  ('company', 0.0070427392761265774),
  ('include', 0.006082210074567497),
  ('base', 0.0060734025655711605),
  ('bat', 0.0060518274354219885),
  ('new', 0.006020978807555916),
  ('use', 0.005314446788306111),
  ('trust', 0.0050430789999759695),
  ('design', 0.004893961018058349),
  ('customer', 0.004743626827403118),
  ('spirit', 0.004647906655508556),
  ('glass', 0.004625134879613373),
  ('package', 0.004434872864287326),
  ('help', 0.004212811197656913)],
 [('flavor', 0.01132143790746434),
  ('available', 0.010078354693321787),
  ('new', 0.009816002691992355),
  ('cocktail', 0.00937133887361428),
  ('music', 0.008974868542425296),
  ('perfect', 0.008391705394403414),
  ('feature', 0.007857226399522247),
  ('whiskey', 0.0078398953

In [ ]:
h = ldaseq.print_topic_times(topic=1)

                               0                                  1   \
0   (flavor, 0.01132143790746434)  (available, 0.010078354693321787)   
1  (flavor, 0.011270723402230912)  (available, 0.010111609429362175)   
2   (flavor, 0.01123546732287486)  (available, 0.010142068927542545)   
3  (flavor, 0.011375770799543753)  (available, 0.010135347500448004)   

                            2                                 3   \
0  (new, 0.009816002691992355)   (cocktail, 0.00937133887361428)   
1  (new, 0.009859020292304736)  (cocktail, 0.009404852043208517)   
2  (new, 0.009905440813336722)  (cocktail, 0.009433176125850463)   
3  (new, 0.009908539661885226)  (cocktail, 0.009409284836375417)   

                              4                                5   \
0  (music, 0.008974868542425296)  (perfect, 0.008391705394403414)   
1   (music, 0.00899694930175626)   (perfect, 0.00843120700735408)   
2  (music, 0.009025013485015781)   (perfect, 0.00847583453022792)   
3  (music, 0.009110165

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
doc_topic, topic_term, doc_lengths, term_frequency, vocab = ldaseq.dtm_vis(time=0, corpus=corpus)
vis_dtm = pyLDAvis.prepare(topic_term_dists=topic_term, doc_topic_dists=doc_topic, doc_lengths=doc_lengths, vocab=vocab, term_frequency=term_frequency)
pyLDAvis.display(vis_dtm)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pyLDAvis/_prepare.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  head(R).drop('saliency', 1)
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:1641: Deprecat